This assignment was copleted using tutorials on the GeeksforGeeks website, and with VSCode's inline generative code completion too.

In [ ]:
%%python3 --version

# create a virtual environment and put it in .venv
%python3 -m venv .venv

# start the environment
%source .venv/bin/activate

%pip install -r requirements.txt
%pip install otter-grader

# start vscode in the current directory if you are using that
%code .

: 

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment1.ipynb")

## Introduction to Pandas and Scikit-Learn

Pandas is a powerful data manipulation and analysis library for Python. It provides data structures like DataFrames and Series that allow for efficient handling of structured data. Pandas is particularly useful for tasks such as reading and writing data in various formats, data cleaning, merging datasets, and performing complex operations on data.

Scikit-learn, on the other hand, is a machine learning library for Python. It provides a wide range of supervised and unsupervised learning algorithms, as well as tools for model selection, evaluation, and preprocessing. Scikit-learn is designed to be user-friendly and efficient, making it a popular choice for both beginners and experienced data scientists.

Together, Pandas and Scikit-learn form a powerful combination for data analysis and machine learning tasks. Pandas is often used to prepare and manipulate data, which can then be fed into Scikit-learn models for training and prediction.

In this assignment, we'll start with the fundamentals of data loading/manipulation in pandas, then move on to basics of scikit-learn.

## Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Let's load the **Census Income** dataset which is available on the [UCI ML Dataset Page](https://archive.ics.uci.edu/dataset/20/census+income).

The goal is to predict whether a person's income was greater than $50K based on 1994 census data.

In [ ]:
df = pd.read_csv('adult.csv')

#### Let's take a quick look at the dataset. We'll use the .head() function to view the first 5 records of the dataset.

In [ ]:
df.head()

## Part 1: Data Loading and pre-processing

### Feel free to refer to the course notes on [pandas](https://tools4ds.github.io/DS701-Course-Notes/02B-Pandas.html) for the following questions.

**Question 1a**: Write a function `get_rows_and_columns` that takes as input a CSV filename, loads this file into a Pandas dataframe, and returns a tuple of the number of rows and columns in the dataframe.

In [ ]:
def get_rows_and_columns(file_path):
    df = pd.read_csv(file_path) 
    return df.shape

In [ ]:
get_rows_and_columns('adult.csv')

In [ ]:
grader.check("q1a")

**Question 1b**: Write a function `compute_missing_percentage` that converts '?' to `pd.NA` and returns the percentage of missing data (i.e., NaNs) for each column in the dataset.

The term `pd.NA` is the way to represent [missing values](https://pandas.pydata.org/docs/reference/missing_value.html) (not available) in Pandas.

In [ ]:
def compute_missing_percentage(df):
    df = df.map(lambda x: pd.NA if x=='?' else x)
    rows,cols = df.shape
    counts = df.count(axis = 0)
    return counts.apply (lambda x: (1-(x/rows))*100)

In [ ]:
compute_missing_percentage(df)

In [ ]:
grader.check("q1b")

**Question 1c**: Write a function `unique_occupation` that returns the number of unique occupation present in the dataset.

In [ ]:
def unique_occupation(df):
    return df['occupation'].nunique()

In [ ]:
unique_occupation(df)

In [ ]:
grader.check("q1c")

**Question 1d**: Write a function `categorical_column_with_max_unique_values` that identifies and returns the index of the **column with maximum number of distinct categorical values** in the dataset.

In [ ]:
def categorical_column_with_max_unique_values(df):
    cats = df.loc[:,['workclass','education','occupation','marital-status','relationship','race','native-country']]
    uniques = cats.nunique(axis = 0)
    return uniques.idxmax()

In [ ]:
categorical_column_with_max_unique_values(df)

In [ ]:
grader.check("q1d")

## Part 2: Exploratory data analysis and visualizations

<!-- BEGIN QUESTION -->

**Question 2a**: Write a function `plot_categorical_distribution` to plot the distribution of the column 'education' as a histogram.

You can use Pandas `.plot()` method for this. Look at the DataFrame `.value_counts` method as well. See class examples for how to add labels and titles.

In [ ]:
def plot_categorical_distribution(df):
    df['education'].value_counts(ascending=True).plot(kind='bar',title='Education Distribution',xlabel='Education Level',ylabel='Count')

In [ ]:
plot_categorical_distribution(df)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2b**: Write a function `plot_age_hours_scatter` that creates a scatter plot of 'age' vs 'hours-per-week', coloring points by 'income'.

You'll want to look at MatPlotLib's `pyplot.scatter()` for this one.

In [ ]:
def plot_age_hours_scatter(df):
    lowdf = df[df['income'] == '<=50K']
    highdf = df[df['income'] == '>50K']
    plt.scatter(lowdf['age'],lowdf['hours-per-week'],s=4,color='blue',label='<=50K')
    plt.scatter(highdf['age'],highdf['hours-per-week'],s=4,color='green',label='>50K')
    plt.title('Age vs Hours per Week')
    plt.xlabel('Age')
    plt.ylabel('Hours per Week')
    plt.legend()
    plt.show()

In [ ]:
plot_age_hours_scatter(df)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2c**: Write a function `plot_income_by_marital_status` which plots a stacked bar chart that shows the proportion of income levels for each 'marital-status' category.

In [ ]:
def plot_income_by_marital_status(df):
    return df.groupby('marital-status')['income'].value_counts().unstack().plot(kind='bar',stacked=True)

In [ ]:
plot_income_by_marital_status(df)

<!-- END QUESTION -->

## Part 3: Advanced Pandas Operations

**Question 3a**: Write a function `education_stats` that returns a dataframe with mean 'age' and median 'hours-per-week' categorized on the 'education' level.

In [ ]:
def education_stats(df):
    res =  df.groupby('education').agg({'age':'mean', 'hours-per-week':'median'})
    res.reset_index(inplace=True)
    return res

In [ ]:
education_stats(df)

In [ ]:
grader.check("q3a")

**Question 3b**: Write a function `calculate_most_popular_occupation` that returns a dataframe of the most popular occupation for each 'native-country' and order them in descending order.

In [ ]:
def calculate_most_popular_occupation(df):
    counts = df.groupby('native-country')['occupation']

    occupations = pd.Series(index=df['native-country'].unique(),dtype=object)
    for country in occupations.index:
       occupations[country] = counts.get_group(country).value_counts().idxmax()
    occupations = occupations.to_frame()
    occupations.columns = pd.Index(['occupation'],dtype='object')
    occupations.index.name = 'native-country'
    occupations.reset_index(inplace=True)
    return occupations

In [ ]:
calculate_most_popular_occupation(df)

In [ ]:
grader.check("q3b")

**Question 3c**: Write a function `workclass_by_income` that returns a dataframe of the top 5 workclass with the highest number of people having income >50K

In [ ]:
def workclass_by_income(df):
     highIncome = df[df['income'] == ">50K"]
     workclass = highIncome['workclass']
     workcounts = workclass.value_counts()
     res = workcounts.to_frame()
     res.columns = pd.Index(['count'],dtype='object')
     res.reset_index(inplace=True)
     return res
     

In [ ]:
workclass_by_income(df)

In [ ]:
grader.check("q3c")

## Scikit-learn

In [ ]:
# MAKE SURE TO RUN THIS CELL!

import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

df_new = df.drop(columns=['native-country', 'fnlwgt']).dropna()

In this part, we'll implement [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) using scikit-learn. Logistic regression is used when trying to predict a binary outcome (0 or 1, True or False, etc.) We will go over the details of logistic regression in details later in the course. 

Here, we will try to predict income (>50k or <= 50k>) and follow standard ML procedures for data pre-processing. You can use scikit-learn's documentation, [the lecture notes on scikit-learn](https://tools4ds.github.io/DS701-Course-Notes/02C-Sklearn.html) or online resources for guidance. 

#### From here on use the 'df_new' variable instead of 'df'

We saw in lecture that models are trained on the 'training' set and evaluated on unseen data on the 'testing' set. The dataset has 'feature' (`X_train`, `X_test`) and the 'outcome' (`y_train`, `y_test`) variables. 

**Question 4a:** Write a function called `split_data` that takes a dataframe as its only parameter, splits it into training and test splits and returns them. Use 20% for the testing set. 

Use `train_test_split` to produce the splits. Provide a `random_state` of 42 for reproducibility.

`split_data` should return 4 things: X_train, X_test y_train and y_test. To do that, you need to pass in the X *and* the y (income column) to `train_test_split`.

In [ ]:
def split_data(df):
    X = df_new.loc[:,df_new.columns != 'income']
    return train_test_split(X,df_new['income'],random_state=42,test_size=0.2)

In [ ]:
grader.check("q4a")

**Question 4b**: Write a function `preprocess_data` that takes X_train, y_train, X_test, and y_test as input (the splits we created earlier!) and does the following:

 - Scale the *numerical* columns using sklearn's `MinMaxScaler` to the range [0,1] for both train and test sets
  
 - Replace "<=50K" with 0 and ">50K" with 1 in both y_train and y_test

 - One-hot encode the categorical columns for both train and test sets. Check the next cell for some hints! 
  
The function should then return the preprocessed X_train, y_train, X_test, and y_test

Refer to the material below and [sklearn course notes](https://tools4ds.github.io/DS701-Course-Notes/02C-Sklearn.html#prepare-the-dataset) for help! 

One hot encoding is a way of turning textual data into numbers, so that models can work with them. 

Pandas has a method called `pd.get_dummies()` that can do one-hot encoding. 

Let's illustrate with an example.  Let's say we are describing 4 people, each
with attribute 'Gender' and 'City' where they reside.

Create a dictionary with 'City' and 'Gender' keys, each with a length 4 list. 

In [ ]:
# Sample DataFrame with categorical columns
data = {'City': ['New York', 'Los Angeles', 'New York', 'Chicago'],
        'Gender': ['Female', 'Male', 'Male', 'Female']}
print(data)

We can give that to Pandas `pd.DataFrame()` and it will create a DataFrame with
a City column and a Gender column.

In [ ]:
df = pd.DataFrame(data) 
print(df)

Now we can call `pd.get_dummies` with the dataframe and it will convert each
categorical column into a set of columns with each category and column entries
of True or False (e.g. 1 or 0).

In [ ]:
df_encoded = pd.get_dummies(df, columns=['City', 'Gender'])
# notice how I'm passing in the columns -- you should do this too! Hint: you wrote a function for this earlier. 

In [ ]:
print(df_encoded)

   Notice how now every `City` value has its own column, and that every row with a city has a 1 for that city (row 1 in the old dataframe has New York for the `City`, and row in the new dataframe has a 1 for `City_New York`). Everywhere else you have a 0. 

In [ ]:
def income2int(income):
    if income == '<=50K':
        return 0
    else:
        return 1
    
def preprocess_data(X_train, X_test, y_train, y_test):
    scaler = MinMaxScaler()
    num_cols = ['age','educational-num','capital-gain','capital-loss','hours-per-week']
    cat_cols = ['workclass','education','occupation','marital-status','relationship','race','gender']

    X_train_num = scaler.fit_transform(X_train[num_cols])
    X_train_num = pd.DataFrame(X_train_num,columns=num_cols,index=X_train.index)
    X_test_num = scaler.transform(X_test[num_cols])
    X_test_num = pd.DataFrame(X_test_num,columns=num_cols,index=X_test.index)  

    X_train_cat = pd.get_dummies(X_train[cat_cols], columns= cat_cols)
    X_test_cat = pd.get_dummies(X_test[cat_cols], columns= cat_cols)

    X_train = pd.concat([X_train_num, X_train_cat], axis=1)
    X_test = pd.concat([X_test_num, X_test_cat], axis=1)

    y_train = y_train.map(income2int)
    y_test = y_test.map(income2int)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = split_data(df_new)
X_train, X_test, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)

In [ ]:
grader.check("q4b")

Let's get to training! Remember, we're trying to predict whether income is more then 50k (>50k), or less than or equal to (<=50k).

**Question 4c:** Write a function called train_model that takes the training splits (X_train and y_train) as its parameters. 

- Initialize the logistic regression model
- Fit it to our data. (Training step)

At the end, return the fitted model. 

You can refer to [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
def train_model(X_train, y_train):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    return reg

In [ ]:
grader.check("q4c")

Let's evaluate the performance of our model! 

**Question 4d:** Write a function called evaluate_model that takes the fitted model and `X_test`, `y_test` as parameters, runs the model on the testing features (`X_test`) and returns the *accuracy score* of the predictions against the ground truth (`y_test`). 

You can refer to [sklearn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics).

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate_model(model, X_test, y_test):
    p = model.predict(X_test)
    return accuracy_score(y_test, p)

In [ ]:
grader.check("q4d")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Execute all cells and save the notebook before submitting.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)